This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'eG1JH4fZ7vP_gq-ZD-i8'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [13]:
# First, import the relevant modules
import requests
import collections
import json
from datetime import datetime

In [50]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?' # Removed my API key to submit

r = requests.get(url)

jsonData = r.json()

print(type(jsonData))
print(jsonData['dataset_data']['data'][0])
print(jsonData['dataset_data']['data'][0][0])
print(type(jsonData['dataset_data']['data'][0][0]))

dataset = jsonData['dataset_data']['data']

working = {}

print(dataset[0])
print(dataset[0][0])

for obj in dataset:
    working[obj[0]] = obj[1:]

data2017 = {}    

for key in working.keys():
    if datetime.strptime(key,'%Y-%m-%d').year == 2017:
        data2017[key] = working[key]
        
print(data2017['2017-10-25'])

<class 'dict'>
['2020-06-26', 88.2, 89.4, 86.65, 88.05, None, 123809.0, 10922723.5, None, None, None]
2020-06-26
<class 'str'>
['2020-06-26', 88.2, 89.4, 86.65, 88.05, None, 123809.0, 10922723.5, None, None, None]
2020-06-26
[45.01, 45.06, 44.7, 45.0, None, 47730.0, 2145697.0, None, None, None]
['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [37]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [66]:
# Calculating the highest and lowest opening prices for the stock in this period

highs = []
for key in data2017.keys():
    if data2017[key][0] != None:
        highs.append(data2017[key][0]) # Values in data2017 are [Open, High, Low, Close, Change, Traded Volume, Turnover, Last Price of the Day, Daily Traded Units, Daily Turnover]

highs.sort()

print(highs[0]) # Lowest opening price = 34.0
print(highs[-1]) # Highest opening price = 53.11

34.0
53.11


In [69]:
# Calculating the largest change in any one day

diff = []
for key in data2017.keys():
    diff.append((key, data2017[key][1] - data2017[key][2])) # Appending difference between daily High and Low, saving as a tuple
    
diff.sort(key=lambda x: x[1]) # Sorting the differential list by the second item in the tuple
print(diff[-1][0]) # Returning the date of the last item in the sorted list of tuples
# Date of largest change = May 11 2017 / 2017-05-11
    

2017-05-11


In [79]:
# Calculating largest change in any two days

diff2 = []

for key in data2017.keys(): # Create a new list of tuples of date / closing price
    diff2.append((key,data2017[key][3]))
    
deltas = []
    
for num in range(1, len(diff2)-1):
    deltas.append((diff2[num-1][0] + ' - ' + diff2[num][0], abs(diff2[num][1] - diff2[num-1][1]))) # Creating tuples of the date range (i.e. between two dates) and the differential of the closing price between them
    
deltas.sort(key = lambda x: x[1]) # Sorting by differential in the tuple

print(deltas[-1]) # Largest change happened between 2017-08-08 - 2017-08-09, 2.559...

('2017-08-09 - 2017-08-08', 2.559999999999995)


In [84]:
# Calculating average daily trading volume

dtv = []

for key in data2017.keys():
    dtv.append(data2017[key][5])
    
print(sum(dtv) / len(dtv)) # Average DTV = 89124.337...


89124.33725490196


In [88]:
# Calculating median trading volume

dtv.sort()
print(dtv[int(len(dtv) / 2)]) # Median trading volume = 76286.0

76286.0
